In [56]:
import sys
print(sys.version)


3.14.0 (tags/v3.14.0:ebf955d, Oct  7 2025, 10:15:03) [MSC v.1944 64 bit (AMD64)]


In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [58]:
pio.renderers.default = 'browser'

In [59]:
df_ple7= pd.read_csv(r"../data/data-1767790355598.csv")
df_ple1 = pd.read_csv(r"../data/data-1767790491093.csv")

df_ple1['maquina'] = 'Plegadora 1'
df_ple7['maquina'] = 'Plegadora 7'

df_ple1 = df_ple1.sort_values('temporal_placa').reset_index(drop=True)
df_ple7 = df_ple7.sort_values('temporal_placa').reset_index(drop=True)


In [ ]:
def preparar_df(df):
    df = df.copy()

    # Timestamp
    df['temporal_placa'] = pd.to_datetime(df['temporal_placa'])
    df['hora'] = df['temporal_placa'].dt.hour
    df['minuto'] = df['temporal_placa'].dt.minute

    # Turnos
    def asignar_turno(hora, minuto):
        t = hora * 60 + minuto

        # Pausas
        if 12*60 <= t < 12*60 + 30:
            return 'ALMUERZO'
        if 22*60 <= t < 22*60 + 30:
            return 'CENA'

        # Turnos
        if 5*60 <= t < 17*60:
            return 'TURNO MAÑANA'
        if 17*60 <= t < 22*60:
            return 'TURNO TARDE'
        if (t >= 22*60 + 30) or (t < 1*60):
            return 'TURNO TARDE'

        return 'FUERA_TURNO'

    df['turno'] = df.apply(
        lambda x: asignar_turno(x['hora'], x['minuto']),
        axis=1
    )

    return df


In [61]:
df_ple1 = preparar_df(df_ple1)
df_ple7 = preparar_df(df_ple7)

df_all = pd.concat([df_ple1, df_ple7], ignore_index=True)

df_all = df_all.sort_values(['maquina','temporal_placa']).reset_index(drop=True)

In [ ]:
# Visualizacion corrientes
import plotly.graph_objects as go
import pandas as pd

fecha = df_all['temporal_placa'].dt.date.min()

horarios = {
    'TURNO_MAÑANA': (f'{fecha} 05:00', f'{fecha} 17:00'),
    'ALMUERZO':     (f'{fecha} 12:00', f'{fecha} 12:30'),
    'TURNO_TARDE':  (f'{fecha} 17:00', f'{fecha} 22:00'),
    'CENA':         (f'{fecha} 22:00', f'{fecha} 22:30'),
    'FUERA_TURNO':  (f'{fecha} 01:00', f'{fecha} 05:00')
}

colores_turnos = {
    'TURNO_MAÑANA': 'rgba(0, 128, 255, 0.08)',   # azul suave
    'TURNO_TARDE':  'rgba(0, 200, 0, 0.08)',     # verde suave
    'ALMUERZO':     'rgba(255, 165, 0, 0.20)',   # naranja
    'CENA':         'rgba(200, 0, 0, 0.20)',     # rojo
    'FUERA_TURNO':  'rgba(120, 120, 120, 0.10)'  # gris suave
}

horarios = {
    k: (pd.to_datetime(v[0]), pd.to_datetime(v[1]))
    for k, v in horarios.items()
}

# Definición de fases
fases = {
    'Fase R': 'corriente_r',
    'Fase S': 'corriente_s',
    'Fase T': 'corriente_t'
}

maquinas = df_all['maquina'].unique()

fig = go.Figure()

colores_maquina = {
    'Plegadora 1': '#1f77b4',  # azul
    'Plegadora 7': '#d62728',  # rojo
}

# -------------------------------------------------
# Trazas: fase × máquina
# -------------------------------------------------
for fase_label, fase_col in fases.items():
    for maq in maquinas:
        g = df_all[df_all['maquina'] == maq]

        fig.add_trace(
            go.Scatter(
                x=g['temporal_placa'],
                y=g[fase_col],
                mode='lines',
                name=f'{maq}',
                line=dict(color=colores_maquina[maq], width=2),
                visible=(fase_label == 'Fase R')  # Fase R visible por defecto
            )
        )

# -------------------------------------------------
# Dropdown para seleccionar fase
# -------------------------------------------------
buttons = []
n_maquinas = len(maquinas)

for i, fase_label in enumerate(fases.keys()):
    visible = [False] * len(fig.data)
    for j in range(n_maquinas):
        visible[i * n_maquinas + j] = True

    buttons.append(
        dict(
            label=fase_label,
            method='update',
            args=[
                {'visible': visible},
                {
                    'title': {
                        'text': f'Corriente vs tiempo – {fase_label}',
                        'x': 0.5,
                        'xanchor': 'center',
                        'font': {'size': 18}
                    }
                }
            ]
        )
    )

    

# -------------------------------------------------
# Layout final
# -------------------------------------------------
fig.update_layout(
      title=dict(
        text='Corriente vs tiempo – Fase R',
        x=0.5,
        xanchor='center',
        font=dict(size=18)
      ),
    yaxis_title='Corriente [A]',
    legend_title='Máquina',
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=0.01,
            y=1.15
        )
    ]
)

for nombre, (inicio, fin) in horarios.items():
    fig.add_vrect(
        x0=inicio,
        x1=fin,
        fillcolor=colores_turnos[nombre],
        opacity=1,
        layer='below',    
        line_width=0,
        annotation_text=nombre.replace('_', ' '),
        annotation_position='bottom left'
    )


fig.show()


In [66]:
# Visualizacion Tensiones
import plotly.graph_objects as go
import pandas as pd
fecha = df_all['temporal_placa'].dt.date.min()

horarios = {
    'TURNO_MAÑANA': (f'{fecha} 05:00', f'{fecha} 17:00'),
    'ALMUERZO':     (f'{fecha} 12:00', f'{fecha} 12:30'),
    'TURNO_TARDE':  (f'{fecha} 17:00', f'{fecha} 22:00'),
    'CENA':         (f'{fecha} 22:00', f'{fecha} 22:30'),
    'FUERA_TURNO':  (f'{fecha} 01:00', f'{fecha} 05:00')
}

colores_turnos = {
    'TURNO_MAÑANA': 'rgba(0, 128, 255, 0.08)',   # azul suave
    'TURNO_TARDE':  'rgba(0, 200, 0, 0.08)',     # verde suave
    'ALMUERZO':     'rgba(255, 165, 0, 0.20)',   # naranja
    'CENA':         'rgba(200, 0, 0, 0.20)',     # rojo
    'FUERA_TURNO':  'rgba(120, 120, 120, 0.10)'  # gris suave
}

horarios = {
    k: (pd.to_datetime(v[0]), pd.to_datetime(v[1]))
    for k, v in horarios.items()
}

# Definición de fases
fases_tension = {
    'Fase R': 'tension_r',
    'Fase S': 'tension_s',
    'Fase T': 'tension_t'
}


maquinas = df_all['maquina'].unique()

fig = go.Figure()

# Colores por máquina (los mismos que corrientes)
colores_maquina = {
    'Plegadora 1': '#1f77b4',  # azul
    'Plegadora 7': '#d62728',  # rojo
}

maquinas = df_all['maquina'].unique()

# -----------------------------------------
# Trazas: fase × máquina (TENSION)
# -----------------------------------------
for fase_label, fase_col in fases_tension.items():
    for maq in maquinas:
        g = df_all[df_all['maquina'] == maq]

        fig.add_trace(
            go.Scatter(
                x=g['temporal_placa'],
                y=g[fase_col],
                mode='lines',
                name=f'{maq}',
                line=dict(
                    color=colores_maquina.get(maq, '#7f7f7f'),
                    width=2
                ),
                visible=(fase_label == 'Fase R')  # arranca en Fase R
            )
        )

# -----------------------------------------
# Dropdown de fases
# -----------------------------------------
buttons = []
n_maquinas = len(maquinas)

for i, fase_label in enumerate(fases_tension.keys()):
    visible = [False] * len(fig.data)

    for j in range(n_maquinas):
        visible[i * n_maquinas + j] = True

    buttons.append(
        dict(
            label=fase_label,
            method='update',
            args=[
                {'visible': visible},
                {
                    'title': {
                        'text': f'Tensión vs tiempo – {fase_label}',
                        'x': 0.5,
                        'xanchor': 'center',
                        'font': {'size': 18}
                    }
                }
            ]
        )
    )

# -----------------------------------------
# Layout
# -----------------------------------------
fig.update_layout(
    title=dict(
        text='Tensión vs tiempo – Fase R',
        x=0.5,
        xanchor='center',
        font=dict(size=18)
    ),
    yaxis_title='Tensión [V]',
    xaxis_title='Tiempo',
    legend_title='Máquina',
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=0.01,
            y=1.15
        )
    ]
)

# -----------------------------------------
# Bandas de turnos y pausas (idéntico a corrientes)
# -----------------------------------------
for nombre, (inicio, fin) in horarios.items():
    fig.add_vrect(
        x0=inicio,
        x1=fin,
        fillcolor=colores_turnos.get(nombre, 'rgba(150,150,150,0.05)'),
        opacity=1,
        layer='below',
        line_width=0,
        annotation_text=nombre.replace('_', ' '),
        annotation_position='bottom left'
    )

fig.show()
